## Optimization using gradient descent

This notebook allows one to do conduct uni- and multivariate linear fits using the gradient descent method. This is a work in progress and does not have all the bugs worked out yet.


In [1]:
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

import sys
sys.path.append('/Users/ondrea/MLandstats/OStats/')
from ostats import ML
from ostats import dplot as dp
#from mpltools import color as colors

In [2]:
data_range = np.random.RandomState(1) #make up some fake data for testing
fakex = 10 * data_range.rand(50)
fakey = 3 * fakex - 5 + data_range.randn(50)


!rm -rf ./data/housing*
!wget -P ./data https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
    
#get some data for testing
!wget -P ./data https://datahub.io/core/global-temp/r/annual.csv

--2021-05-31 17:47:04--  https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘./data/housing.csv’

housing.csv         100%[===================>]   1.36M  6.53MB/s    in 0.2s    

2021-05-31 17:47:04 (6.53 MB/s) - ‘./data/housing.csv’ saved [1423529/1423529]

--2021-05-31 17:47:04--  https://datahub.io/core/global-temp/r/annual.csv
Resolving datahub.io (datahub.io)... 172.67.157.38, 104.21.40.221
Connecting to datahub.io (datahub.io)|172.67.157.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pkgstore.datahub.io/core/global-temp/annual_csv/data/a26b154688b061cdd04f1df36e4408be/annual_csv.csv [fol

In [3]:
year, tempa = np.loadtxt('./data/annual.csv', skiprows=1, delimiter=",", usecols=(1,2), unpack=True)
year = np.array([int(y) for y in year])/np.max(year); tempa=np.array(tempa)+1

In [4]:
year = 1 + (year - np.mean(year))/(np.max(year)-np.min(year))

In [5]:
#year

In [6]:
housing_data = pd.read_csv('./data/housing.csv')

In [7]:
housing_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [8]:
housing_data = housing_data[housing_data.median_house_value != 500001] #a bunch of values are set to this
housing_data = housing_data.sample(n=250) #smaller sample
housing_data.median_house_value = housing_data.median_house_value/100000
housing_data.total_bedrooms = housing_data.total_bedrooms/1000

In [9]:
housing_data.median_house_value.max()

4.955

In [10]:
housing_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
1162,-121.52,39.51,30.0,3085.0,0.610,1688.0,575.0,2.3340,0.722,INLAND
3130,-117.95,35.08,1.0,83.0,0.015,32.0,15.0,4.8750,1.417,INLAND
3983,-118.63,34.18,32.0,1646.0,0.242,697.0,233.0,6.6689,4.330,<1H OCEAN
6648,-118.14,34.17,52.0,2687.0,0.600,1716.0,544.0,2.7201,2.057,<1H OCEAN
4786,-118.32,34.03,47.0,1734.0,0.453,1272.0,438.0,3.1731,1.215,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
11371,-117.98,33.71,26.0,1905.0,0.373,1098.0,368.0,4.8611,2.296,<1H OCEAN
13378,-117.62,34.15,16.0,13556.0,1.704,5669.0,1668.0,6.5138,3.115,INLAND
3166,-118.91,35.27,29.0,1401.0,0.317,1344.0,306.0,2.0921,0.614,INLAND
6033,-117.73,34.08,28.0,5173.0,1.069,3502.0,954.0,3.8438,1.308,INLAND


In [11]:
x_house = housing_data['median_house_value'].to_numpy()
y_house = housing_data['total_bedrooms'].to_numpy()

In [12]:
def plotfit(x, beta):
    funct = beta[-1][0]+x*beta[-1][1]
    return(funct)

def plotfit2(x, beta):
    functs=[]
    points = np.linspace(0,len(beta),10)
    points = [int(x) for x in points]
    for i in range(len(points)-1):
        functs.append(beta[points[i]][0]+x*beta[points[i]][1])
    return(functs)

### Fake data

In [13]:
ifig=1;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
theta_fake, J_fake = ML.GradDes_Regression(x=fakex, y=fakey, gamma=0.01, l=1)
plt.scatter(fakex,fakey, c='darkred',  s=15)
nlines = 10
for i in range(nlines):
    plt.plot(fakex,plotfit2(fakex,beta=theta_fake)[i-1], color=dp.ColorGradient(stop=(11, 0, 59), n=nlines)[i],lw=1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

715


/Users/ondrea/MLandstats/OStats/ostats/ML.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  stop_condition = (np.abs(theta[1] - store_theta[-1][1])/store_theta[-1][1] < epsilon) and (np.abs(theta[0] - store_theta[-1][0])/store_theta[-1][0] < epsilon)


### Housing data

In [ ]:
theta_house_normal=ML.Normal_Linear_Regression(x=x_house, y=y_house, l=1)
ynormal = np.array(theta_house_normal[1])*x_house + np.array(theta_house_normal[0])

theta_house, J_house = ML.GradDes_Regression(x=x_house, y=y_house, gamma=0.001, epsilon = 0.001, l=1)
theta_house_alt, J_house_alt = ML.GradDes_Regression(x=x_house, y=y_house, gamma=0.01,epsilon = 0.001, l=1)

In [ ]:
ifig=2;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)

plt.scatter(x_house, y=y_house, c='g',  s=15, label='data')
plt.plot(x_house,ynormal, c='b')
nlines = 10
for i in range(nlines):
    plt.plot(x_house,plotfit2(x_house,beta=theta_house)[i-1],\
             color=dp.ColorGradient(stop=(0, 200, 90), n=nlines)[i],lw=1)
    plt.plot(x_house,plotfit2(x_house,beta=theta_house_alt)[i-1],\
             color=dp.ColorGradient(stop=(140, 20, 90), n=nlines)[i],lw=1)


In [ ]:
### Climate data
theta_climate, J_climate = ML.GradDes_Regression(x=year, y=tempa, gamma=0.01, l=1, epsilon=0.0001, theta_init = np.array([0.5,0.5]))
theta_climate_normal=ML.Normal_Linear_Regression(x=year, y=tempa, l=1)
ynormal_climate = np.array(theta_climate_normal[1])*year + np.array(theta_climate_normal[0])

In [ ]:
ifig=3;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)

plt.scatter(year, y=tempa, c='orange',  s=15)
plt.plot(year,ynormal_climate, c='b')
nlines = 10
for i in range(nlines):
    plt.plot(year,plotfit2(year,beta=theta_climate)[i-1],\
             color=dp.ColorGradient(stop=(31, 80, 40), n=nlines)[i],lw=1)

## Now take a look at convergence by plotting $J(\theta)$ vs $(\theta)$ or $J(\theta)$ vs n iter

In [ ]:
theta_house, J_house = ML.GradDes_Regression(x=x_house, y=y_house, gamma=0.01, epsilon = 0.001, l=1)
theta_house_alt, J_house_alt = ML.GradDes_Regression(x=x_house, y=y_house, gamma=0.001,epsilon = 0.000001, l=1)

In [ ]:
#For the two instances of housing data
ifig=4;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
theta1 = [x[0] for x in theta_house]
theta1_alt = [x[0] for x in theta_house_alt]
theta2 = [x[1] for x in theta_house]
theta2_alt = [x[1] for x in theta_house_alt]

niter = range(len(theta1))
niter2 = range(len(theta1_alt))

plt.plot(J_house, theta1, color = 'k')
plt.plot(J_house_alt, theta1_alt, marker='.', ls='',color='k', alpha=0.2)
plt.plot(J_house, theta2, color = 'r')
plt.plot(J_house_alt, theta2_alt, marker='.',ls='', color = 'r', alpha=0.1)

In [ ]:
ifig=5;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.plot(niter, J_house, color = 'k')
plt.plot(niter2, J_house_alt, marker='_', color='k')

In [ ]:
#For the two instances of housing data
ifig=6;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.plot(theta1, theta2, c='b', lw=3)
plt.plot(theta1_alt, theta2_alt, c='y')

In [ ]:
ifig=7;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.plot(niter,theta1)
plt.plot(niter,theta2)

plt.plot(niter2,theta1_alt)
plt.plot(niter2,theta2_alt)